 # Who Watches The Watchmen? Finding Spyplanes with Autoencoder Neural Networks

One of the weirder datasets on Kaggle is the 'spyplanes' dataset. It consists of a bunch of flight data from 2015 and was originally used as part of a Buzzfeed News article where the author reckoned he could detect spyplanes apart from other aircraft using machine learning. His analysis worked by training a model on flight data for known FBI and US Security Services aircraft and then looking for other aircraft with similar behaviour.

I think there is another approach to this problem.

Surveillance-aircraft are, by definition, supposed to be difficult to identify. By comparing aircraft behaviour with that of known government aircraft, we risk missing many other planes which may behave drastically differently to those we manually identified. Rather than training a model on a known dataset (aka supervised learning), I decided it is more appropriate to scour the whole dataset of flight data for unusual behaviour (aka unsupervised learning). In other words, I'm going to look for aircraft which look 'weird'...otherwise known as outliers. To do this I'm going to use an autoencoder neural network.

# The Dataset

Full details of the dataset used are available at https://www.kaggle.com/jboysen/spy-plane-finder
    
It consists of modified flight data from 2015. As this project was more about finding an excuse to use an autoencoder, I chose a pretty convenient dataset which has already been engineered for features. Were I using raw flight data, I'd probably have performed the feature engineering differently, but that's not interesting now.

Each entry includes an aircraft with a number of distance/time based metrics, to give an idea of the typical flight pattern. Also included is the aircraft type and transponder info and number of observations/flights seen over the period.

The original author also included a small directory of aircraft which he believed were spy aircraft (as they were operated by the FBI or US Defence Department) and ones which he believed were not. In total he identified 97 probably spy aircraft and 500 non-spy aircraft. He used this 'known' dataset to train up his model. I am going to used it for model validation, but not in the normal sense. Remember, we don not know that all of the 97 aircraft are actually spyplanesand any of the 500 ones could be too.

#Processing the dataset

Rather than training the model on a dataset and then testing it with known results, a kind of hybrid approach was used. A model was trained on a subset of the main dataset and then tested on the table of known aircraft. Since we do not know how many of the labels on the 'known' aircraft are correct, the performance was only really an estimate. The model was then applied to the rest of the main dataset to give candidate spyplanes. Normally we'd just run a model on the raw dataset and let it tell us which entries were the outliers, but in this case we have the luxury of some 'sort of' labelled data.

First the necessary libraries are installed and the main dataset imported. A second table called 'labelled_data' is created. This contains the 'known' spyplanes and non-spyplanes and will be used to test the model. These aircraft are removed from the main dataset.

In [3]:
#import libraries
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix

#import dataset
df=pd.read_csv("C:\\Users\\Alex\\Documents\\datasets\\spy-plane-finder\\planes_features.csv")

#convert type column to integer (there are probably better approaches to use here)
df['type']=df['type'].astype('category').cat.codes

#import labelled aircraft
test_ident = pd.read_csv("C:\\Users\\Alex\\Documents\\datasets\\spy-plane-finder\\train.csv")

#use the labelled data as a train/test set (note the different context of testing to normal for this model)
labelled_data=df[df['adshex'].isin(test_ident['adshex'])]
labelled_data=pd.merge(labelled_data,test_ident,on=['adshex','adshex'])
labelled_data['type']=labelled_data['type'].astype('category').cat.codes
labelled_data=labelled_data.drop(['adshex'],axis=1)

df=df[~df['adshex'].isin(test_ident['adshex'])]
df_adshex=df['adshex']
df=df.drop(['adshex'],axis=1)

10% of the aircraft from the main dataset are then removed and used to build a training set. Note, the data was scaled between 0-1:

In [4]:
#use 10% of the input data as a training set
df,train_set=train_test_split(df,test_size=0.1,random_state=57)

#also consider adding some of the actual data to the train/test set to improve the size
test_set = labelled_data

#save test set labels for later
test_set_labels=test_set['class']
#get number of positive classes in test set
test_set_positives=len(test_set[test_set['class']=='surveil'])
test_set = test_set.drop(['class'], axis=1)

#convert to array and normalise
train_set = preprocessing.MinMaxScaler().fit_transform(train_set.values)
test_set = preprocessing.MinMaxScaler().fit_transform(test_set.values)


# Applying A Model

The aim was to find an algorithm which would identify outliers in the data. For this dataset, aircraft which behave oddly may well be surveillance aircraft. I used an autoencoder neural network to do this, here's how they work:

INSERT DESCRIPTION HERE

To do this with Keras/Tensorflow we first define the layers of the network. The input and output layers must be the same size as the data, with a node for each attribute. The intermediate layers wre half the size, to allow the network to compress the records as described above.

In [5]:
#define layers
input_dim = test_set.shape[1]
encoding_dim = int(input_dim/2)

input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim, activation="tanh", 
                activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder = Dense(int(encoding_dim / 2), activation="relu")(encoder)
decoder = Dense(int(encoding_dim / 2), activation='tanh')(encoder)
decoder = Dense(input_dim, activation='relu')(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

Next the model was run and the best performing model saved. Note, all the parameters like the optimiser and the loss function. These can be changes along with the number and size of the layers. This process is called hyperparameter tuning and we seek to find the best combination of hyperparameters to obtain the best results. For this excercise, some random paramaters were tried and the best selection chosen. Normally I'd tune the parameters far more extensivley but since this is just a demo I didn't want to get too bogged down with this part.

Note, the model was only run on the 10% of aircraft which were used as the training set. Ideally I'd train it only on aircraft which were known not to be spyplanes. This would make it optimised for reconstructing 'normal' aircraft patterns only and unusual aircraft not seen in the training set would be reconstructed with a high degree of error. I figured very few of the training set aircraft would br spyplanes so this is probably not a huge issue.

In [ ]:

nb_epoch = 100
batch_size = 50
autoencoder.compile(optimizer='Adamax', 
                    loss='mean_squared_error', 
                    metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath="model.h5",
                               verbose=0,
                               save_best_only=True)
tensorboard = TensorBoard(log_dir='./logs',
                          histogram_freq=0,
                          write_graph=True,
                          write_images=True)
history = autoencoder.fit(train_set,train_set,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(test_set,test_set),
                    verbose=1,
                    callbacks=[checkpointer, tensorboard]).history

autoencoder=load_model('model.h5')

Now it's time to test the model on the dataset of 'known' aircraft. Remember, I am not convinced that all of those labelled 'spyplanes' are indeed spyplanes. Similarly, many of the aircraft labelled 'normal' could well be spylplanes. I was therfore not expecting great performance. To classify aircraft, I ensured the model only labelled the top 97 anomalous entries as spyplanes, because that is how many were labelled as such in the original 'known' aircraft dataset. 

In [ ]:
#predict on testing set
predictions=autoencoder.predict(test_set)
rmse = pow(np.mean(np.power(test_set - predictions, 2), axis=1),0.5)
error_table = pd.DataFrame({'reconstruction_error': rmse,
                        'actual_class': test_set_labels})
#we know how many entries have a positive in the test set. Take this number and label the predictions with the highest rmse (ie the outliers) with the positiveclass prediction
error_table['predicted_class'] = np.where(error_table['reconstruction_error'] >= min(error_table.nlargest(int(test_set_positives),'reconstruction_error', keep='first')['reconstruction_error']), 'surveil', 'other')

#get confusion matrix
print(confusion_matrix(error_table['actual_class'],error_table['predicted_class']))


When I initially ran the script, 31 of the 97 spyplanes were correctly identified. By tuning the parameters, I managed to improve this to 54, which gives the following confusion matrix:

|              |   normal   |  spyplane     |
|--------------|------------|---------------|
|   normal     |    457     |       43      |
|--------------|------------|---------------|
|   spyplane   |     43     |      54       |

At first glance this might look like a not-too-great result but I was actually pretty impressed how well it performed - it found 54 of the 97 aircraft which were known to be government operated. Let's remember that the algorithm is identifying unusual aircraft just by the fact they fly in an irregular pattern - it is not using any known data to correlate observed aircraft with known spyplanes.

Let's also remember that some of the aircraft identified as spyplanes which were labelled as 'normal' may indeed be spyplanes. The test data relies on accurate labelling of aircraft which is impossible.

Let's have a look at the reconstruction error for each of the aircraft in the test set. This is a measure of how much of an outlier each aircraft was - recall the top 97 outliers were taken to be spyplanes:

In [ ]:
#plot error
groups=error_table.groupby('actual_class')
fig,ax=plt.subplots()
for name, group in groups:
    ax.plot(group.index,group.reconstruction_error,marker='o', ms=3.5, linestyle='',
            label= name)